# Bottom up inventory generation

This script attempts to generate a bottom-up inventory as an example for the users.

We will generate the emissions for few sectors using various data sources.
The goal here is not to create a perfect inventory but to 
* show various examples how emissions can be calculated
* show different needs in term of spatial and temporal modelling of emissions
* how these can be integrated in emiproc 

We decided here to create the inventory for the
[Corvo island](https://en.wikipedia.org/wiki/Corvo_Island) in the Azores
as it is a small island so it will not cost a lot of processing and will be
quite fun to do.
It is a volcanic island with a population of around 384 people (2021).
The biggest activites are agriculture and tourism.

In [ ]:
import logging

import geopandas as gpd
import pandas as pd
import requests
from shapely.geometry import LineString, Point, Polygon

from emiproc.utils.osm import osm_json_to_gdf


logging.basicConfig(level=logging.INFO)

## Download Corvo from Openstreetmap

We will download everything available from Openstreetmap for the island of Corvo.
We will then see if we can extract some useful information from it.

Note: If you would do that for a larger place you might get a lot of data to process.
It could be a better idea to select only what you want.

This firt part of the code is a bit more technical but if you look at the result, it should be quite easy to understand.

In [ ]:

# Define the bounding box

# Corvo
# bbox = (39.5813, -31.3172, 39.8117, -30.7782)
# Faial
bbox = (38.5004, -28.8535, 38.6598, -28.5885)


In [ ]:


bbox = (39.5813, -31.3172, 39.8117, -30.7782)
url = "https://overpass-api.de/api/interpreter"
query = f"""[out:json];
nwr({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
out;"""
response = requests.get(url, params={"data": query})
data = response.json()

In [ ]:
data_by_id = {x["id"]: x for x in data["elements"]}

In [ ]:

gdf = osm_json_to_gdf(data, extract_tags=['landuse', 'building', 'highway'])

gdf = gdf.set_index("id")


### Landuse tag

The landuse tag gives us a lot of information on the island.

Here we will have a look to all the landuse shapes availables.

In [ ]:
gdf_landuse = gdf.loc[gdf['landuse'].notnull()]
gdf_landuse.explore('landuse')

From the map, we can see we have various landuse types.

We can get more information about land use on the [Openstreetmap wiki](https://wiki.openstreetmap.org/wiki/Key:landuse).

In particular, here we will start with doing the emissions from the various argicutural activities.
So we will focus on the following landuse types:
* farmland (An area of farmland used for tillage (cereals, vegetables, oil plants, flowers)) 
* farmyard (An area of land with farm buildings like farmhouse, dwellings, farmsteads, sheds, stables, barns, equipment sheds, feed bunkers, etc. plus the open space in between them and the shrubbery/trees around them.)
* meadow (A meadow or pasture: land primarily vegetated by grass and non-woody plants, mainly used for hay or grazing)

In [ ]:

gdf_farmland = gdf.loc[gdf["landuse"] == "farmland"]
gdf_farmland.explore()

### Emissions from cows 

As you probably know, cows are a major source of methane emissions by doing what is referred as "Enteric Fermentation".
They contribute globally to around 14% of the total emissions of methane.

As Corvo is a small highland with cattle as one of the main activities, this will be a very large source.


To estimate the emissions from cows we need an emission factor, which is the amount of the ghg produced by a cow per unit of time.
Let's use the table 10.11 from the [IPCC 2019 guidelines for the emissions factors](https://www.ipcc-nggip.iges.or.jp/public/2019rf/vol4.html).

We assume that half cows are dairy cows: $ 126 \frac{kg_{CH4}}{head\ year}$ and the other half are beef cows: $ 52 \frac{kg_{CH4}}{head\ year}$.

To find the number of cows, the only source is not so great, as just [a report from a tourist](http://v2.travelark.org/travel-blog-entry/modernnomad67/36/1467554508) that says about 1500 cows are on the island.



In [ ]:
enteric_fermentation_CH4 = (126 + 52) / 2 * 1500
# We need to convert the crs to a metric one
gdf_farmland['area'] = gdf_farmland.geometry.to_crs(epsg=3763).area
total_area_farmland = gdf_farmland['area'].sum()
# We will assign emissions based on the size of the fields 
gdf_farmland['enteric_fermentation_CH4'] = enteric_fermentation_CH4 * gdf_farmland['area'] / total_area_farmland


### Emissions for the Power plant 

If you research electricity production in Corvo, you will see a lot about how they will change to 100% renewable energy.
Indeed for such a small island, it is quite easy to do so, and is also a project (though a bit old since from 2011) that 
makes a good marketing.
There is even a scientific publication about it: [Integration of wind and hydrogen technologies in the power system of Corvo island, Azores: A cost-benefit analysis](https://doi.org/10.1016/j.ijhydene.2010.12.074).

For this task, we will assume that there is a single power plant in Corvo, which is a diesel power plant.

The publication teaches us the following:
* The island has an annual demand of approximately 1084 MWh and peak of 182 kW. 
* The total fuel (diesel) consumption in Corvo is 288,051 L/yr and the total CO2 emissions produced yearly on the island are 758,532 kg/year
* Emissions of other pollutants in kg/year: CO: 1872, UHC: 207, PM10: 141, SO2: 1523, NOx: 16707

These values are probably overestimated for today since they already installed some {V plnat}


With this we can easily write the data.

In [ ]:
emissions_powerplant = {
    # CO: 1872, UHC: 207, PM10: 141, SO2: 1523, NOx: 16707
    "CO2": 758532,
    "CO": 1872,
    "UHC": 207,
    "PM10": 141,
    "SO2": 1523,
    "NOx": 16707,
}
# From maps
location_powerplant = Point(-31.0975, 39.7111)

#### Time profiles for the energy demand

The paper above said they have received the energy demand profiles for the island.
However in our current case we lack of this data so we will generate some data with a first guess.

In [ ]:
from emiproc.profiles.temporal_profiles import DailyProfile, WeeklyProfile, MounthsProfile

# They show a morning peakd and an evening peak due to the consumption of the appliances
daily_profile = DailyProfile(5*[0.01] + 4*[0.04] + 3*[0.030] + 6* [0.055] + 4*[0.085] + 2*[0.015])

# For mounthly profiles, we have to do things differently, as we know that many tourists come in the summer
monthly_profile = MounthsProfile([
    0.055, 0.055, 0.055, 0.065, 0.09, 0.12, 0.13, 0.13, 0.11, 0.08, 0.055, 0.055
])
energy_profiles = [daily_profile, monthly_profile]
plt.plot(daily_profile.ratios.T)

### Emissions from heating

Here again we are a bit lucky: A case study for replacing heating systems with heat pumps and solar thermal collectors in Corvo was done: https://doi.org/10.1016/j.renene.2014.06.046  

The study gives us the following information:
* DHW (Domestic hot water) butane gas demand: 335 MWh/year or 3315 bottles/year
* 60% of the energy is for DHW
* A guess of daily profiles for the DHW demand are given

We can estimate that each house is equipped with a boiler and that depending on the size of the house, the demand will be different.

Also we need to convert the energy demand to CO2 emissions. We will use the emission factors from table 2.5 
of https://www.ipcc-nggip.iges.or.jp/public/2006gl/pdf/2_Volume2/V2_2_Ch2_Stationary_Combustion.pdf 

In kg/TJ:
* CO2: 56100
* CH4: 5
* N2O: 0.1

We still need to convert MWh to TJ.


To get the building locations, we can use openstreetmap again and look for the building tag.

In [ ]:
# kg/TJ
total_emissions = {
    "CO2": 56100,
    "CH4": 5,
    "N2O": 0.1,
}
# Convert from MWh to TJ 
gas_demand = 335 * 3.6e-3 # MWh/year to TJ/year

total_emissions = {k: v * gas_demand  for k, v in total_emissions.items()}

In [ ]:
# We can check which buildings we have
gdf_landuse = gdf.loc[gdf["building"].notnull()]
gdf_landuse.explore("building")

In [ ]:
# We will use only the domestic sectors, probably on real data you will have more than these categories
residential_buildings = ['yes', 'house', 'apartments', 'detached', 'residential']
gdf_residential = gdf.loc[gdf["building"].isin(residential_buildings)].copy()
gdf_residential['area'] = gdf_residential.geometry.to_crs(epsg=3763).area
# Now we will assign emissions based on the size of the buildings
for substance, emission in total_emissions.items():
    gdf_residential[substance] = emission * gdf_residential['area'] / gdf_residential['area'].sum()
gdf_residential.explore("CO2")

### Traffic emissions

OSM has all the roads registerd in its system.

However traffic data will not be easy to obtain.
Most inhabitants don't have cars and the island is so small that the traffic is very low.
One main road goes to the top of the crater and is used by tourists.
Other roads are probably used by the farmers.
For the tourists basically they can use a taxi to go to the top of the crater, or to move from the port or the airport to the hotel.

In [ ]:
gdf_traffic = gdf.loc[gdf['highway'].notnull()]
gdf_traffic.explore('highway')